In [1]:
import pandas as pd
import sys, os
sys.path.append("../main/")
import match
import ZI
import matplotlib.pyplot as plt

# Cleaning data

Per poter simulare il LOB tramite tramite il modello ZI ho bisogno delle seguenti informazioni:

1. Tipo di ogni ordine nel book.
2. Quota a cui è avvenuto ogni ordine.
3. Volume di ogni ordine.

In aggiunta per il modello MTY ho bisogno anche del:

4. Prezzo di ogni ordine.
5. Segno di ogni ordine.

Purtroppo queste informazioni non sono disponibili, in quanto ho solamente un dataframe con lo stato del LOB alle 10 migliori quote del bid e dell'ask e un dataframe con le informazioni di sopra relative solamente ai trades.

Queste informazioni però possono essere direttamente ricavate dallo stato del book.

Per prima cosa carico il LOB relativo ad un giorno di trading per ELE-GER attraverso la funzione load_data della libreria match:

### Input

1. filepath: str
    - Path assoluto del file .csv da aprire
2. del_time: [True, False]  (default = False)
    - Se True elimina la prima e ultima ora di trading dal dataframe.
3. del_spread: [True, False] (default = False)
    - Se True elimina dal LOB tutti gli stati in cui risulta che lo spread è <= 0
4. start_month: [True,False] (default = False)
    - Se True aggiunge una colonna con i secondi passati dall'inizion del mese (Per fare matching degli ordini è fondamentale che start_month sia True).

### Output

1. df: pd.DataFrame
    - DataFrame dello stato del LOB.


In [2]:
# carica dataframe LOB del 01/10/2021
DIR = "../data/energia/LOB_ottobre21/LOB_ottobre21/LOB_Germany_Baseload_2022_20211001.csv"
# per funzionare correttamente devo utilizzare l'absolute path
filepath = os.path.abspath(DIR)
df_o = match.load_data(filepath, start_month = True)

In [3]:
df_o.head()

,Datetime,BidPrice_0,BidVolume_0,AskPrice_0,AskVolume_0,BidPrice_1,BidVolume_1,AskPrice_1,AskVolume_1,BidPrice_2,...,BidVolume_8,AskPrice_8,AskVolume_8,BidPrice_9,BidVolume_9,AskPrice_9,AskVolume_9,MidPrice,Spread,Seconds
0,2021-10-01 06:00:01.630,10850.0,1.0,14000.0,1.0,5200.0,1.0,0.0,0.0,4800.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12425.0,3150.0,21601.630
1,2021-10-01 06:00:19.222,10850.0,1.0,14000.0,1.0,9380.0,1.0,0.0,0.0,5200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12425.0,3150.0,21619.222
2,2021-10-01 06:02:37.526,13000.0,1.0,14000.0,1.0,10850.0,1.0,0.0,0.0,9380.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13500.0,1000.0,21757.526
3,2021-10-01 06:03:29.627,13000.0,1.0,14000.0,2.0,10850.0,1.0,0.0,0.0,9380.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13500.0,1000.0,21809.627
4,2021-10-01 06:03:56.548,13000.0,1.0,13900.0,1.0,10850.0,1.0,14000.0,1.0,9380.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13450.0,900.0,21836.548


Sia per il modello MTY che per il modello ZI l'order-flow è composto da 3 processi:

1. Limit Order (LO)
2. Market Order (MO)
3. Cancellazioni (C)

Nel LOB relativo ai dati energetici è presente anche un altro tipo di ordine:

4. Update (UP)

Un trader che ha piazzato un LO può decidere di aumentare o abbassare il prezzo del proprio LO, esempio:



In [4]:
df_o.loc[12:13, ["BidPrice_0", "BidPrice_1"]]

,BidPrice_0,BidPrice_1
12,12000.0,11997.0
13,12500.0,11997.0


Come si può vedere nella cella di sopra il prezzo dell'ordine alla migliore quota del bid è aumentato di 5€, mentre il prezzo dell'ordine alla seconda migliore quota è rimasto uguale.

Questo significa che questo ordine deve per forza essere un updarte, perché se si fosse trattato realmente di un nuovo LO il BidPrice_1 sarebbe dovuto essere 120€.

A questo punto prima di andare avanti bisogna per prima cosa trovare tutti gli update e poi bisogna modellizzarli come una cancellazione + un LO.

Per trovare tutti gli update ho utilizzato la funzione clean_data:

### Input

1. df: pd.DataFrame
    - Dataframe contenente lo stato del book alle 10 migliori quote dell'ask e del bid. 
    - E' importante che il DataFrame contenga anche le seguenti colonne: ["Quote", "Type", "Sign","Price", "Volume"]

### Output
1. None


Al DataFrame di partenza vengono però aggiunte le seguenti informazioni:

1. Volume degli ordini.
2. Prezzo degli ordini.
3. Segno dell'ordine.
4. Quota a cui è stato fatto l'ordine.
5. Il tipo di ordine:
    - 0.0: Ordine eseguito al di fuori delle 10 migliori quote dell'ask e del bid
    - Limit: Limit order.
    - Market/Cancel: Ordine che può essere sia un trade che una cancellazione.
    - Canc/Update: Update in cui è stato aumentato il prezzo dell'ordine.
    - Lim/Update: Update in cui è stato abbassato il prezzo dell'ordine.

In [5]:
# Aggiungo le seguenti quote al dataframe degli ordini:
# 1. Quote, 2.Type, 3. Sign, 4.Volume, 5.Price

df_o["Quote"] = df_o["BidVolume_0"] * 0 - 999
df_o["Type"] = df_o["BidVolume_0"] * 0
df_o["Sign"] = df_o["Type"]
df_o["Price"] = df_o["Type"]
df_o["Volume"] = df_o["Type"]

# Pulisco dati 
match.clean_data(df_o)

# Printo il tipo di ordini
print("Tipo di ordini nel DataFrame dopo il processo di cleaning:")
df_o["Type"].value_counts()

Cleaning data...

Tipo di ordini nel DataFrame dopo il processo di cleaning:


Limit            4584
Market/Cancel    4376
Lim/Update       1492
0.0              1139
Canc/Update       619
Name: Type, dtype: int64

A questo punto dopo aver trovato tutti gli update li modellizzo come una cancellazione + un LO, utilizzando la funzione update_df:

### Input

1. df: pd.DataFrame
    - DataFrame "pulito" con la funzione clean_data.

### Output
1. final_df: pd.DataFrame
    - DataFrame in cui ogni update è stato modellizzato come una cancellazione + un LO.

In [6]:
df_1 = match.update_df(df_o)

Modifying update orders...



Per capire bene cosa fa la funzione update_df riprendo come esempio le righe 12-13 del DataFrame originale:

In [7]:
df_o.loc[12:13, ["BidPrice_0", "BidPrice_1"]]

,BidPrice_0,BidPrice_1
12,12000.0,11997.0
13,12500.0,11997.0


Tra la riga 12 e la riga 13 manca una cancellazione. Quello che fa la funzione update_df in questo caso è cancellare dapprima l'ordine da 120€ per poi piazzare un LO di 125€: 

In [8]:
df_1.loc[12:14, ["BidPrice_0", "BidPrice_1"]]

,BidPrice_0,BidPrice_1
12,12000.0,11997.0
13,11997.0,11994.0
14,12500.0,11997.0


A questo punto riutilizzo la funzione clean data per calcolare il prezzo, volume, segno, quota e tipo di ogni ordine.

In [9]:
# Pulisco dati 
match.clean_data(df_1)

# Printo il tipo di ordini
print("Tipo di ordini nel DataFrame dopo il processo di cleaning:")
df_1["Type"].value_counts()

Cleaning data...

Tipo di ordini nel DataFrame dopo il processo di cleaning:


Limit            6698
Market/Cancel    6484
0.0              1139
Name: Type, dtype: int64

L'ultima cosa da fare è differenziare le cancellazioni dai MO. 
Per prima cosa importo il DataFrame con le informazioni relative ai trades attraverso la funzione load_trade_data:


### Input

1. filepath: str
    - Path assoluto del file .csv da aprire
4. start_month: [True,False] (default = False)
    - Se True aggiunge una colonna con i secondi passati dall'inizion del mese (Per fare matching degli ordini è fondamentale che start_month sia True).

### Output

1. df: pd.DataFrame
    - DataFrame contenente le informazioni relative ai trades.

In [10]:
# carica dataframe trade del 01/10/2021
DIR_1 = "../data/energia/trade_ottobre2021_nuovo/trade_ottobre2021/trades_Germany_Baseload_2022_20211001.csv"
filepath = os.path.abspath(DIR_1)
df_t = match.load_trade_data(filepath, start_month = True)
df_t.head()

,DateTime,Action,Volume,Price,AggressorAction,AggressorBroker,Seconds
0,2021-10-01 06:26:39+00:00,Insert,1.0,13040.0,Sell,EEX,23199.000
1,2021-10-01 06:28:56+00:00,Insert,1.0,13150.0,Sell,EEX,23336.000
2,2021-10-01 06:33:56.257000+00:00,Update,1.0,13155.0,Sell,Tradition,23636.257
3,2021-10-01 06:34:09.287000+00:00,Update,1.0,13150.0,Sell,Griffin Markets Europe SAS,23649.287
4,2021-10-01 06:35:01+00:00,Insert,1.0,13065.0,Sell,EEX,23701.000


Infine per fare il match degli ordini utilizzo la funzione matching:

### Input

1. order_df: pd.DataFrame
    - DataFrame degli ordini pulito con la funzione clean_data.
2. trade_df: pd.DataFrame
    - DataFrame dei trades.
3. criterion: {time", "time price volume sign", "time price volume", "time price", "time volume",
    "time sign", "time price sign", "time volume sign", "best matching"} (default = "time"):
    - Criterio considerato per fare matching.
4. time_interval: int (default = 5)
    - Considero come candidati per il matching tutti gli ordini in un intervallo di +- time interval secondi da un trade.
    
### Output
1. match_df: pd.DataFrame
    - DataFrame in cui ho differenziato trades e cancellazioni.

In [11]:
matched_data = match.matching(df_1, df_t, criterion = "time price sign", time_interval = 4)

Matching orders...
Number of orders without match : 226, out of : 743


In [12]:
matched_data["Type"].value_counts()

Limit     6698
Cancel    5999
0.0       1139
Market     485
Name: Type, dtype: int64